In [1]:
import pandas as pd

In [2]:
# Executing this cell does some magic
%load_ext autoreload
%autoreload 2

In [3]:
full_df = pd.read_parquet("data/full_df.parquet")


In [4]:
%cd ..

d:\OneDrive\Personal_Things\My Website\My website HTML\backend\supplychainAPI


In [5]:
weather_df = pd.read_parquet("./data/weather_01012026.parquet")

In [16]:
weather_df.head()

,date,snowfall,rain,precipitation,wind_speed_10m,wind_gusts_10m,cloud_cover_low,cloud_cover,temperature_2m,apparent_temperature,surface_pressure,relative_humidity_2m,pressure_msl,queried_airport_code
0,2018-01-01 00:00:00-05:00,0.0,0.0,0.0,16.575644,30.960001,100.0,100.0,3.65,-0.640734,1017.892456,86.149536,1025.199951,ABY
1,2018-01-01 01:00:00-05:00,0.0,0.0,0.0,16.575644,31.680000,100.0,100.0,2.85,-1.760402,1018.070068,78.900909,1025.400024,ABY
2,2018-01-01 02:00:00-05:00,0.0,0.0,0.0,18.175545,34.919998,100.0,100.0,2.30,-2.882902,1018.353088,68.517944,1025.699951,ABY
3,2018-01-01 03:00:00-05:00,0.0,0.0,0.0,18.003599,35.639999,99.0,99.0,1.60,-3.862596,1018.831116,59.252598,1026.199951,ABY
4,2018-01-01 04:00:00-05:00,0.0,0.0,0.0,18.003599,36.000000,72.0,96.0,0.90,-4.777263,1018.911560,52.844017,1026.300049,ABY


In [17]:
display(len(full_df['Origin'].unique()), len(weather_df['queried_airport_code'].unique()))

388

118

In [18]:
airports = full_df['Origin'].unique().tolist()
collected_airports = weather_df['queried_airport_code'].unique().tolist()
remaining_airports = list(set(airports) - set(collected_airports))

In [20]:
len(remaining_airports)

270

In [7]:
from openmeteo_api.src.openmeteoapi.WeatherData import Weather
from openmeteo_api.src.openmeteoapi.APICaller import OpenMeteoAPICaller
import os
from dotenv import load_dotenv

In [21]:
BATCH_SIZE = 10

def barch_list(items, batch_size: int) -> list[int]:
    for i in range(0, len(items), batch_size):
        yield items[i:i + batch_size]
BATCHES = list(barch_list(remaining_airports, BATCH_SIZE))

In [22]:
len(BATCHES)

27

In [58]:
from tqdm import tqdm
import time

length = len(BATCHES)
start_date = "2018-01-01"
end_date = "2022-12-31"
api_caller_weather = OpenMeteoAPICaller()

collected_airports = []

for i in tqdm(range(0, length)):
    airport_list = BATCHES[i]
    print(airport_list)
    if len(airport_list) == 0: break
    w = Weather(
        api_caller=api_caller_weather,
        airport_code=airport_list,
        code_type="iata",
        start_date=start_date,
        end_date=end_date,
    )

    for attempt in [20, 60, 120]:
        try:
            response = w.to_hourly_dataframe()
            df = response["data"]
            dfs.append(df)
            collected_airports.extend(df['queried_airport_code'].unique().tolist())
            break
        except Exception as e:
            if "limit" in str(e).lower():
                time.sleep(attempt)
            else:
                raise

    time.sleep(5)

  0%|          | 0/19 [00:00<?, ?it/s]

['LIT', 'PQI', 'ROW', 'RDD', 'EKO', 'GPT', 'LYH', 'ABI', 'SHD', 'TYR']
Fetching weather data from: https://archive-api.open-meteo.com/v1/archive


  0%|          | 0/19 [02:00<?, ?it/s]


KeyboardInterrupt: 

In [23]:
def barch_list(items, batch_size: int) -> list[int]:
    for i in range(0, len(items), batch_size):
        yield items[i:i + batch_size]
BATCHES = list(barch_list(airports, BATCH_SIZE))

In [ ]:
BATCHES

[['ABY', 'ATL', 'MOB', 'BUF', 'DFW'],
 ['BTV', 'CVG', 'LGA', 'CHO', 'EWN'],
 ['MCI', 'MGM', 'MSP', 'DCA', 'FAY'],
 ['OAJ', 'STL', 'CWA', 'DTW', 'RDU'],
 ['SRQ', 'AEX', 'AUS', 'CSG', 'EVV'],
 ['IND', 'CLE', 'PIA', 'TRI', 'BGR'],
 ['IAD', 'PWM', 'TYS', 'GSO', 'AVL'],
 ['BTR', 'MKE', 'CMH', 'GRR', 'GSP'],
 ['BIS', 'ORD', 'ROC', 'CHA', 'TVC'],
 ['FSM', 'JAX', 'BOS', 'GRB', 'TUL'],
 ['VLD', 'CRW', 'IAH', 'XNA', 'JAN'],
 ['CHS', 'ROA', 'GFK', 'SYR', 'MDT'],
 ['SAT', 'BMI', 'LFT', 'BNA', 'PHL'],
 ['BQK', 'ORF', 'GNV', 'OMA', 'TPA'],
 ['SAV', 'BWI', 'MEM', 'MBS', 'SDF'],
 ['LEX', 'CLT', 'DAY', 'MOT', 'FAR'],
 ['CAE', 'DSM', 'PIT', 'DHN', 'MHT'],
 ['ILM', 'SGF', 'ELM', 'BDL', 'MCO'],
 ['RIC', 'SHV', 'MSY', 'HSV', 'TLH'],
 ['LAN', 'LIT', 'GRK', 'MSN', 'ATW'],
 ['HRL', 'HPN', 'CAK', 'EWR', 'RSW'],
 ['BQN', 'PBI', 'FLL', 'SLC', 'PSE'],
 ['JFK', 'SJC', 'LAS', 'LGB', 'LAX'],
 ['SFO', 'SJU', 'HOU', 'DEN', 'SEA'],
 ['STT', 'PVD', 'PDX', 'SAN', 'RNO'],
 ['ALB', 'PHX', 'OAK', 'PSP', 'SMF'],
 ['STX', 'BU

In [26]:
remaining_batches = BATCHES[-4:]
remaining_batches

[['PAE', 'PIR', 'ATY', 'BFM', 'XWA'],
 ['CDB', 'DDC', 'DEC', 'SHR', 'RIW'],
 ['JST', 'VCT', 'ALS', 'FOD', 'MCW'],
 ['ILG', 'TBN', 'BIH']]

In [27]:
for i in tqdm(range(len(remaining_batches))):
    print(remaining_batches[i])
    if len(remaining_batches[i]) == 0: break
    w = Weather(
        api_caller=api_caller_weather,
        airport_code=remaining_batches[i],
        code_type="iata",
        start_date=start_date,
        end_date=end_date,
    )

    for attempt in range(3):
        try:
            df_weather = w.to_hourly_dataframe()
            dfs.append(df_weather)
            break
        except Exception as e:
            if "limit" in str(e).lower():
                time.sleep(60)
            else:
                raise

    time.sleep(5)

  0%|          | 0/4 [00:00<?, ?it/s]

['PAE', 'PIR', 'ATY', 'BFM', 'XWA']
Fetching weather data from: https://archive-api.open-meteo.com/v1/archive


 25%|██▌       | 1/4 [00:16<00:48, 16.28s/it]

['CDB', 'DDC', 'DEC', 'SHR', 'RIW']
Fetching weather data from: https://archive-api.open-meteo.com/v1/archive


 50%|█████     | 2/4 [00:31<00:31, 15.81s/it]

['JST', 'VCT', 'ALS', 'FOD', 'MCW']
Fetching weather data from: https://archive-api.open-meteo.com/v1/archive
Fetching weather data from: https://archive-api.open-meteo.com/v1/archive


 75%|███████▌  | 3/4 [01:47<00:43, 43.13s/it]

['ILG', 'TBN', 'BIH']
Fetching weather data from: https://archive-api.open-meteo.com/v1/archive
Fetching weather data from: https://archive-api.open-meteo.com/v1/archive


100%|██████████| 4/4 [02:59<00:00, 44.76s/it]


In [43]:
weather_df = pd.read_parquet("./data/weather.parquet")

In [54]:
weather_df_more = pd.concat(dfs + [weather_df], ignore_index=True)

In [56]:
len(weather_df_more['queried_airport_code'].unique())

198

In [57]:
weather_df_more.to_parquet("./data/weather_01012026.parquet")

In [1]:
import requests

url = "https://prod.api.market/api/v1/aedbx/aerodatabox/flights/Number/DL27/2026-01-20?dateLocalRole=Departure&withAircraftImage=false&withLocation=false&withFlightPlan=false"
headers = {
    "accept": "application/json",
    "x-api-market-key": "cmjnqufj50009ky04q88ac1a9",
}

response = requests.get(url, headers=headers)
print(response.json())

[{'greatCircleDistance': {'meter': 11510400.07, 'km': 11510.4, 'mile': 7152.23, 'nm': 6215.12, 'feet': 37763779.77}, 'departure': {'airport': {'icao': 'KATL', 'iata': 'ATL', 'name': 'Atlanta Hartsfield Jackson', 'shortName': 'Hartsfield Jackson', 'municipalityName': 'Atlanta', 'location': {'lat': 33.6367, 'lon': -84.4281}, 'countryCode': 'US', 'timeZone': 'America/New_York'}, 'scheduledTime': {'utc': '2026-01-21 04:55Z', 'local': '2026-01-20 23:55-05:00'}, 'terminal': 'I', 'quality': ['Basic']}, 'arrival': {'airport': {'icao': 'RKSI', 'iata': 'ICN', 'name': 'Seoul Incheon', 'shortName': 'Incheon', 'municipalityName': 'Seoul', 'location': {'lat': 37.4691, 'lon': 126.451}, 'countryCode': 'KR', 'timeZone': 'Asia/Seoul'}, 'scheduledTime': {'utc': '2026-01-21 20:45Z', 'local': '2026-01-22 05:45+09:00'}, 'predictedTime': {'utc': '2026-01-21 20:16Z', 'local': '2026-01-22 05:16+09:00'}, 'terminal': '2', 'quality': ['Basic']}, 'lastUpdatedUtc': '2025-11-25 16:18Z', 'number': 'DL 27', 'status': 